In [1]:
from Crypto.Cipher import AES
import binascii
from base64 import b64encode, b64decode

In [2]:
def init_global():
    #global parameters
    p = 353
    g = 3
    return p, g

In [3]:
def generate_keys(PRA, PRB):
    p, g = init_global()
    print('Global parameters \n', 'p:', p , 'g:', g)
    print('Private keys \n', 'Alices private key:', PRA, '\n Bobs private key:', PRB)
    
    #Generating public keys
    PUA = (g**PRA)%p
    PUB = (g**PRB)%p
    print('Alice sends her public key to Bob:', PUA)
    print('Bob sends his public key to Alice:', PUB)
    
    #Generating shared keys
    Alice_Kab = (PUB**PRA)%p
    Bob_Kab = (PUA**PRB)%p
    print('Alice generates a shared key:', Alice_Kab)
    print('Bob generates a shared key:', Bob_Kab)
    if Alice_Kab == Bob_Kab:
        shared_key = Alice_Kab
    return shared_key

In [ ]:
def BBS_generator(Kab):
    seed = Kab
    key_length = 64        #Fixed because this is the only length that will give an acceptable input for AES
    M = 43*47              #p and q, randomly chosen primes, high values to get a wider cycle
    key = ''
    for i in range(0, key_length): #BBS formumla realization
        seed = (seed**2)%M
        bit = seed & 1           #Bit selection method; keeping only the LSB
        key += str(bit)
    key = hex(int(key, 2))    #Transforming the binary key to hex-dec
    return key

In [5]:
def encrypt(message, key):
    key = key[2:]            #Removing "0x" from the key
    key = bytes(key, 'utf-8')  #Re-casting the key to bytes instead of nibbles
    aes = AES.new(key, AES.MODE_ECB) #Creating AES object
    plain = message
    block_size = 16
    ciphertext = b''
#Splitting the message into 16 byte pieces. Each piece is then encrypted individually
    while len(plain) > 0:
        block = plain[0:block_size]
        if len(block)<block_size:
            block = block + ' '*(block_size-len(block))
        ciphertext += aes.encrypt(block)
        plain = plain[block_size:]
    
    return b64encode(ciphertext).decode()     #b64encode to re-format the cipher into text instead of hex-dec 



In [6]:
def decrypt(ciphertext, key):
    key = key[2:]
    ciphertext = b64decode(ciphertext.encode())
    key = bytes(key, 'utf-8')
    aes = AES.new(key, AES.MODE_ECB)
    plaintext = aes.decrypt(ciphertext)
    return plaintext.decode()

In [ ]:
#Random secret keys
PRA = eval(input('Alice, choose your private key:'))
PRB = eval(input('Bob, choose your private key:'))

shared_key = generate_keys(PRA, PRB)
BBS_key = BBS_generator(shared_key)
print('Using BBS generator, Alice and Bob generates a more secure key: ', BBS_key)

a = True
while a == True:
    b = True
    while b == True:
        EorD = eval(input('Do you want to encrypt or decrypt? \n 1 for encrypt; 2 for decrypt: '))

        if EorD == 1:
            message = input('Enter the message you want to encrypt: ')
            cipher = encrypt(message, BBS_key)
            print('The encrypted message is: ',cipher)
            b = False
        elif EorD == 2:
            message = input('Enter the message you want to decrypt: ')
            plain = decrypt(cipher, BBS_key)
            print('The depcrypted message is: ',plain)
            b = False
        else:
            print('Incorrect choice entered. Try again!')
            
    repeat = eval(input('Encrypt or decrypt again? 1 for yes; anything else for no: '))
    if repeat != 1:
        print('Session ended.')
        a = False



Alice, choose your private key:142
Bob, choose your private key:87
Global parameters 
 p: 353 g: 3
Private keys 
 Alices private key: 142 
 Bobs private key: 87
Alice sends her public key to Bob: 72
Bob sends his public key to Alice: 339
Alice generates a shared key: 201
Bob generates a shared key: 201
BBS generated key, binary:  0111100000101111111000110111001101000000101010100001111001101110
Using BBS generator, Alice and Bob generates a more secure key:  0x782fe37340aa1e6e
Do you want to encrypt or decrypt? 
 1 for encrypt; 2 for decrypt: 1
Enter the message you want to encrypt: Hello Bob
The encrypted message is:  INPxfI3FiLoD3Ioph6ZuWw==
Encrypt or decrypt again? 1 for yes; anything else for no: 1
Do you want to encrypt or decrypt? 
 1 for encrypt; 2 for decrypt: 2
Enter the message you want to decrypt: INPxfI3FiLoD3Ioph6ZuWw==
The depcrypted message is:  Hello Bob       
Encrypt or decrypt again? 1 for yes; anything else for no: 1
Do you want to encrypt or decrypt? 
 1 for encryp